# Spark Streaming with Python and Kafka

## Preparing the Environment

For Spark Streaming to be able to access Kafka, we need to have it load an external package. We are using a modified version of the great [docker-stacks](https://github.com/jupyter/docker-stacks). The updated dockerfile downloads this package, and we just need to pass it to `PYSPARK_SUBMIT_ARGS` before starting Spark from within Jupyter.

To run the code in Jupyter, you can put the cursor in each cell and press Shift-Enter to run it one cell at a time -- or you can use menu option `Kernel` -> `Restart & Run All`. When a cell is executing you'll see a `[*]` next to it, and once the execution is complete this changes to `[y]` where `y` is execution step number. Any output from that step will be shown immediately below it.


In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark-streaming-kafka-0-8-assembly_2.11-2.1.0.jar pyspark-shell'

### Import dependencies
We will import the necessary pySpark modules for Spark, Spark Streaming, and the utils for interascting with Kafka from within Spark. We also need the python json module for parsing the inbound twitter data, and a few helpful arithmetic functions

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
from operator import add, sub

### Create Spark context
Let's start the Spark context in local mode using all the available cores

In [ ]:
sc = SparkContext('local[*]')

### Create Streaming Context

Using the spark context above, create a streaming context with a batch interval of 5 seconds.

See the [API reference](http://spark.apache.org/docs/2.1.0/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext) and [programming guide](http://spark.apache.org/docs/2.1.0/streaming-programming-guide.html#initializing-streamingcontext) for more details. 

In [ ]:
batchInterval = 10
ssc = StreamingContext(sc, batchInterval)

For performing windowed computations Spark Streaming requires a checkpoint directory for saving intermediate data. This uses local disk to store the checkpoint data. In a Production deployment this would be on resilient storage such as HDFS.

In [ ]:
ssc.checkpoint('/tmp')

### Connect to Kafka

Using the native Spark Streaming Kafka capabilities, we use the streaming context from above to connect to the Kafka container. We use the advertised hostname `kafka` and the exposed port for the broker `9092`. We are also subscribing to the previously used topic `Twitter.live`.

For more information see the [documentation](http://spark.apache.org/docs/latest/streaming-kafka-0-8-integration.html).

In [ ]:
kafka_host = 'kafka:9092'
topic = 'Twitter.live'
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {'bootstrap.servers': kafka_host})

The messages returning from kafka are of the form (key, serialized json value), we want to extract the values


In [ ]:
tweets = kafkaStream.map(lambda kv: json.loads(kv[1]))

### Count number of tweets in the batch

Streaming data is continuous and needs to be batched to process. Spark Streaming divides the data stream into batches of 10 seconds (or the chosen batch interval) called [`DStreams`](http://spark.apache.org/docs/2.1.0/api/python/pyspark.streaming.html#pyspark.streaming.DStream), which internally is a sequence of RDDs (Resilient Distributed Datasets). Your Spark Application processes the RDDs using Spark APIs, and the processed results of the RDD operations are returned in batches.

_![](https://www.mapr.com/sites/default/files/blogimages/sparkstream2-blog.png)
Image credit: [mapr blog](https://www.mapr.com/blog/spark-streaming-hbase)_

The object provides a few native functions: 

* [`count`](http://spark.apache.org/docs/2.1.0/api/python/pyspark.streaming.html#pyspark.streaming.DStream.count) - gets the number of messages in the batch.
* [`pprint`](http://spark.apache.org/docs/2.1.0/api/python/pyspark.streaming.html#pyspark.streaming.DStream.pprint) - prints a defualt number of 10 messages to the output

_Note that nothing gets written to output from the Spark Streaming context and descendent objects until the Spark Streaming Context is started, which happens later in the code_

The `map` function will apply a given function to each RDD inside the Dstream. In the code below if simply formats the text so it's more easily identifiable on the output.

Finally, the `lambda` function is a Python construct to create an anonymous function (so that we don't have to define it beforehand). You can read more about it [here](https://pythonconquerstheuniverse.wordpress.com/2011/08/29/lambda_tutorial/).

In [ ]:
tweets.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

### Extract Author name from each tweet

We already had a look at the JSON structure for the tweets. We're going to analyse tweets by author, which is accessible in the json structure we built under `user.screen_name`. 

The `authors_dstream` will hold just the author's screen name for each tweet in the original DStream.

In [ ]:
authors_dstream = tweets.map(lambda tweet: tweet['user']['screen_name'])
authors_dstream.pprint()

### Extract hashtags from each tweet
It is likely we won't have repeated authors in the short batch interval we picked, let's look at a more interesting use case: hashtags.

The hashtags are within the tweet text, so we split each tweet into word tokens (by spaces), apply a flatmap to have a single list of tokens per dstream, and filter for those starting with a hash.

The `hashtags_dstream` will hold just the tags occurring in the original DStream.

In [ ]:
hashtags_dstream = tweets.flatMap(lambda tweet: tweet['text'].split(" ")).filter(lambda word: word.startswith("#"))

### Count the number of tags per batch
We'll use a convenience function `CountByValue` to aggregate tags in the DStream. 

In [ ]:
hashtags_counts = hashtags_dstream.countByValue()
hashtags_counts.pprint()

### Print the top 5 tags
The `pprint()` above might miss out on the actual top tags, since they are not sorted.

If you try and use the `sortBy` function directly against the DStream you get an error: 

    'TransformedDStream' object has no attribute 'sortBy'
    
This is because sort is not a built-in [DStream](http://spark.apache.org/docs/2.1.0/api/python/pyspark.streaming.html#pyspark.streaming.DStream) function, we use the [`transform`](http://spark.apache.org/docs/2.1.0/streaming-programming-guide.html#transform-operation) function to access [`sortBy`](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.sortBy) from pySpark. 


In [ ]:
hashtags_counts_sorted = hashtags_counts.transform(
    lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
hashtags_counts_sorted.pprint(5)

### The word count example
Let's take the most common distributed processing example shall we?

I'm using line separators below to make the code more readable.

In [ ]:
tweets.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x: x[1], ascending=False))\
    .pprint()

### Filter tweets by word
We can also use `filter` effectively to reduce the DStream by a given condition. Here let's filter out any tweets that do not have the word `IoT` in them.

*Note*: this will match without consideration for word boundaries

In [ ]:
tweets_filtered = tweets.filter(lambda tweet: 'iot' in tweet['text'].lower())
tweets_filtered.map(lambda tweet: tweet['text']).pprint(1)

### Windowed stream processing
Often it might be useful to perform computations in sliding windows that are larger than the batch. The window size must be a multiple of the batch duration that we worked with above. So instead of counting how many tweets there are every batch (say, 10 seconds), we could instead count how many there are per minute - the _window_ interval. We can perform this count potentially every time the batch runs; how frequently we do the count is known as the _slide_ interval.

_![](http://spark.apache.org/docs/latest/img/streaming-dstream-window.png)
Image credit, and more details about window processing, [here](http://spark.apache.org/docs/latest/streaming-programming-guide.html#window-operations)._

In [ ]:
count_windowed = tweets.countByWindow(60,10).map(lambda x:('Tweets total (One minute rolling count): %s' % x))
count_windowed.pprint()

## Start the streaming context

Having defined the streaming context, now we're ready to actually start it! When you run this cell, the program will start, and you'll see the result of all the `pprint` functions above appear in the output to this cell below. If you're running it outside of Jupyter (via `spark-submit`) then you'll see the output on stdout.

The `timeout` will deliberately cancel the execution after two minutes.

In [ ]:
ssc.start()
ssc.awaitTermination(timeout=120)